In [0]:
%python
## Import libraries
import pyspark as ps
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *
import pandas as pd
import json

In [0]:
class GetSuggestion:
  def __init__(self):
    self.query = "Query About to fill"
    self.sql_df = pd.DataFrame()
    self.data_json = {"Data":[]}    
  
  def logger(message):
    print(message)
    
  def set_query_string(self):
    query = """WITH t(NGEBOX_RECORD_ID, zip_cd, id, account_vod__c) AS 
(select distinct request_table.NGEBOX_RECORD_ID, terr.zip_cd, usr.id, request_table.account_vod__c from com_us_alyt_ngebox.ib_NGEBox_Suggestion_Requests  request_table 
left join com_us_alyt_ngebox.METADATA_USECASE met on request_table.usecase_id = met.USECASE_ID
join com_us_alyt_ngebox.lkp_prod_salesforce  lkup on lkup.Prod_Nm = met.Prod_Nm
left join (select distinct sf_cd,rpt_sales_org_key, zip_cd, geo_cd from com_us_hub.ref_zip_terr
  where curr_ind = 'Y' and level = 'Territory' ) terr
on request_table.hcp_zip_code = terr.zip_cd
and terr.sf_cd = lkup.sf_cd
left join com_us_hub.ref_emp_assignment assign
on terr.geo_cd = assign.geo_code
and current_date() between assignment_start_date and assignment_end_date
inner join com_us_hub.ref_iengage_user usr
on assign.emp_id = usr.EMPLOYEENUMBER and usr.isactive = 1)

merge into com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed process_table using (select distinct tb2.id,tb1.NGEBOX_RECORD_ID as NB_record_processed,tb2.NGEBOX_RECORD_ID as NB_record_t, tb4.hist_retention,tb1.suggest_external_id_vod__c  from com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed tb1
inner join t tb2 on tb1.NGEBOX_RECORD_ID = tb2.NGEBOX_RECORD_ID
left join com_us_lake.rep_suggestion_vod__c tb3 on tb3.SUGGESTION_EXTERNAL_ID_VOD__C = tb1.suggest_external_id_vod__c
inner join com_us_alyt_ngebox.metadata_usecase tb4 on tb1.usecase_id = tb4.usecase_id
where (tb3.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) 
or (tb3.SUGGESTION_EXTERNAL_ID_VOD__C is not null and tb3.DISMISSED_VOD__C = 0)) source_table on (process_table.NGEBOX_RECORD_ID = source_table.NB_record_processed and process_table.NGEBOX_RECORD_ID = source_table.NB_record_t and process_table.suggest_external_id_vod__c = source_table.suggest_external_id_vod__c and process_table.created_date >= date_add(current_date,-source_table.hist_retention))
when matched then
update set process_table.ownerid = source_table.id;"""
    self.query = query

  def run_query_and_set_sqldf(self):
    query_list = self.query.split(";")
    self.sql_df = []
    for curr_query in query_list:
      if curr_query.strip() == "":
        continue
      sql_df = sqlContext.sql(curr_query).collect() # type(sql_df) -> dict
      self.sql_df = self.sql_df + sql_df
    
  def format_sqldf_to_json(self):
    sql_output_rowlist = []
    for curr_row in self.sql_df: 
      sql_output_rowlist.append (curr_row.asDict(True))
    sql_df_new = pd.DataFrame(sql_output_rowlist)
    sql_jsonstr = sql_df_new.to_json(orient="records") # type(result) -> str
    sql_jsonparse = json.loads(sql_jsonstr) # type(parsed_json) -> list
    nb_json = {"data":sql_jsonparse} # type(metadata_json) -> dict
    self.nb_json = nb_json
    
  def construct_microservice_response(self):
    resp = {
      "nge_response": {
        "status": 200,
        "body": self.nb_json
      }
    }
    self.response = resp

In [0]:
%python
data_obj = GetSuggestion()
data_obj.set_query_string()
data_obj.run_query_and_set_sqldf()
data_obj.format_sqldf_to_json()
data_obj.construct_microservice_response()
dbutils.notebook.exit(
  data_obj.response
)

{'nge_response': {'status': 200, 'body': {'data': [{'num_affected_rows': 138, 'num_updated_rows': 138, 'num_deleted_rows': 0, 'num_inserted_rows': 0}]}}}